# Get Data from Strava API
I obtain data from Strava on all cycling efforts on the segment "Diablo - North Gate to Summit," a popular Bay Area road cycling segment that ascends the north side of Mount Diablo.

In [60]:
# import requisit libraries
import requests
import time
import pickle
import sys
import os
import strava_api # my Strava API credentials

### Collect Data
I use Strava's API to obtain data on the Mt Diablo south gate-to-summit segment. See details about Strava's authentication protocol [here](https://strava.github.io/api/v3/oauth/). 

For security reasons, I keep my API access token hidden. Anyone wishing to reproduce this analysis should create a file within this working directory called `strava_api.py` and within that file populate a string variable `access_token` with their own access token.

#### Parameters

In [55]:
# populate some general parameters
extra_headers = {'Authorization' : 'Bearer %s' % strava_api.access_token}
api_base_url = 'https://www.strava.com/api/v3/'
api_segment_url = api_base_url + 'segments/%d'
api_segment_all_efforts_url = api_segment_url + '/all_efforts'

per_page = 200 # Strava max

segment_id = 656860 # "Diablo - North Gate to Summit"

# for writing data to file
segment_fname = 'segment_%d.p' % segment_id
all_efforts_fname = 'all_efforts_%d.p' % segment_id

#### Segment Data
First, I'll get data on the segment itself.

In [56]:
# get some data on the segment itself
segment_r = requests.get(api_segment_url % segment_id, headers=extra_headers)
n_efforts = segment_r.json()['effort_count']

#### Segment Efforts
Strava returns a max of 200 efforts per "page," but there are over 23,000 efforts on this segment. Thus, I iterate through and combine the pages of efforts.

In [67]:
start = time.time()
all_efforts = []
# do we want feedback on progress and elapsed time?
verbose = 0 

# if we already have the data pickled, load the file. otherwise, request effort data from Strava API 
if os.path.isfile(all_efforts_fname):
    sys.stdout.write('Unpickling all efforts from %s\n' % all_efforts_fname)
    all_efforts = pickle.load(open(all_efforts_fname, 'rb'))
else:
    for i in range(1, round(2 + n_efforts / per_page)):
        if verbose:
            sys.stdout.write('Requesting page %d\n' % i)
        r = requests.get(api_segment_all_efforts_url % segment_id, headers=extra_headers, params={'per_page' : per_page, 'page' : i})

        if r.status_code != 200:
            sys.stderr.write('Error, received code %d for summary request %d\n' % (r.status_code, i))
        else:
            all_efforts.extend(r.json())
            #time.sleep(2) # Make sure do not hit Strava rate limiting
        
# how much time elapsed (sec) during the loop?
end = time.time()
elapsed = end-start
if verbose:
    sys.stdout.write('\nTotal time elapsed (seconds): %f\n' % elapsed)

Unpickling all efforts from all_efforts_656860.p


### Save Data
Write data to pickle files.

In [57]:
# save segment data
pickle.dump(segment_r, open(segment_fname, 'wb'))

# save effort data
pickle.dump(all_efforts, open(all_efforts_fname, 'wb'))